In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import numpy as np
from scipy import stats
import seaborn as sns
from functools import reduce
pd.options.mode.chained_assignment = None 
from statistics import mean
from Profile_Generation import *
from plotting import *
from analysis import *
import warnings
warnings.simplefilter('ignore', np.RankWarning)
from scipy.optimize import curve_fit
import matplotlib.patches as patches
from scipy.optimize import curve_fit
from openpyxl import load_workbook
import math
from scipy.optimize import OptimizeWarning
warnings.simplefilter("ignore", OptimizeWarning)
import timeit
from sklearn.metrics import mean_squared_error
from matplotlib.pyplot import cm
from matplotlib.patches import Patch
from scipy import stats
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
%%time
# ~5mins

# Execute CURATE without pop tau
execute_CURATE(pop_tau_string='_by_body_weight_2')

In [ ]:
%%time
# Perform CV
five_fold_cross_val_results, five_fold_cross_val_results_summary = find_pop_tau_with_CV()
execute_CURATE_and_update_pop_tau_results('CV', five_fold_cross_val_results_summary, five_fold_cross_val_results)

# Perform LOOCV
five_fold_cross_val_results, five_fold_cross_val_results_summary = find_pop_tau_with_LOOCV()
execute_CURATE_and_update_pop_tau_results('LOOCV', five_fold_cross_val_results_summary, five_fold_cross_val_results)

In [8]:
a = pd.Series([1,2,3])
b = pd.Series([4,5,6])

c = pd.DataFrame({'a': a, 'b':b})

# Create extra row with empty cell in b
c.loc[len(c), 'a'] = ""

c['a'] = c['a'].shift(1)

c

,a,b
0,NaN,4.0
1,1.0,5.0
2,2.0,6.0
3,3.0,NaN


In [36]:
df = dat.copy()
df[df.day<3]

,day,response,dose_mg,patient,ideal,body_weight,dose
0,2,2.4,0.5,84,True,9.70,0.051546
35,2,5.6,1.0,117,False,11.70,0.085470
47,2,3.4,3.0,118,True,48.00,0.062500
59,2,2.3,0.5,120,True,9.38,0.053305
83,1,21.6,4.0,121,False,50.20,0.079681
84,2,12.9,0.0,121,False,50.20,0.000000
96,2,7.7,1.5,122,False,11.00,0.136364
117,1,3.6,1.0,123,True,11.10,0.090090
118,2,5.6,2.0,123,True,11.10,0.180180
141,1,2.3,0.5,126,False,14.00,0.035714


upper quartile: 2.80

distribution within extreme prediction errors: 3.95 [IQR 3.32 -  5.38]


,patient,pred_day,abs_deviation
1562,84,6,4.8
7527,114,7,4.6
7528,114,8,5.0
7529,114,9,5.5
7530,114,10,3.8
7532,114,12,3.8
7537,114,17,5.6
15445,118,5,4.0
23910,120,7,4.1
23913,120,10,3.9


In [24]:
df = values_by_dosing_strategy()
df

Repeated:
N = 11
    First day to achieve therapeutic range:     4.0 [IQR     3.0 -     6.5] days
    Days in therapeutic range (%):     28.57 [IQR     13.33 -     35.90]

Distributed:
N = 5
    First day to achieve therapeutic range:     8.0 [IQR     6.0 -     9.0] days
    Days in therapeutic range (%):     18.18 [IQR     12.50 -     21.43]


,patient,dosing_strategy,first_day_therapeutic_range,perc_therapeutic_range
0,84,repeated,7.0,38.461538
1,114,distributed,17.0,18.181818
2,117,repeated,3.0,33.333333
3,118,repeated,10.0,16.666667
4,120,distributed,8.0,12.500000
5,121,repeated,4.0,7.692308
6,122,repeated,5.0,28.571429
7,123,distributed,6.0,10.000000
8,125,repeated,3.0,57.142857
9,126,repeated,7.0,10.000000
